In [55]:
from random import seed, shuffle
from collections import defaultdict
from itertools import chain

def number_to_card(n):
    """将牌号转换为Unicode字符"""
    suit_map = defaultdict(
        lambda: 0x1F0B1,  # 默认：红桃
        {
            0: 0x1F0B1,  # 红桃
            1: 0x1F0C1,  # 方块
            2: 0x1F0D1,  # 梅花
            3: 0x1F0A1   # 黑桃
        }
    )
    rank_offset = [0, 6, 7, 8, 9, 10, 12, 13]  # A,7,8,9,10,J,Q,K
    
    suit, rank = n // 8, n % 8  # 计算花色和牌面值
    
    return chr(suit_map[suit] + rank_offset[rank])  # 返回对应的Unicode字符

def display_board(placed_card):
    result = []
    for line in placed_card:
        result.append("\t" + "\t".join(line))
    return result

def update(placed_card, waiting_to_match, num):
    waiting_to_match.remove(num)
    row, col = 12 - num % 13, num // 13 - 1
    placed_card[row][col] = number_to_card(num)
    
    if 0<row<6:
        waiting_to_match.append(num+1)
    elif 6<row<12:
        waiting_to_match.append(num-1)


def run_one_round(number, result, waiting_to_match, cards, placed_card):
    number_to_word = {
        1: "First",
        2: "Second", 
        3: "Third"
    }
    result.append(f"Starting {number_to_word[number]} round...")
    result.append("")

    already_checked = []
    for num in cards[::-1]:
        current_card = cards.pop()
        if current_card in waiting_to_match:
            result.append("Placing card from top of stack of cards left 😊️")
        else:
            result.append("Cannot place card from top of stack of cards put aside ☹️")
            already_checked.append(current_card)
            
        result.append("]"* len(cards))
        if already_checked:
            result.append("["*(len(already_checked)-1)+ number_to_card(already_checked[-1]))
        else:
            result.append("")

        if current_card in waiting_to_match:
            update(placed_card, waiting_to_match, num)
            result.extend(display_board(placed_card))
            result.append("")
            
            while already_checked and already_checked[-1] in waiting_to_match:
                result.append("Placing card from top of stack of cards put aside 😊️")
                result.append("]" * len(cards))
                update(placed_card, waiting_to_match, already_checked.pop())
                if already_checked:
                    result.append("["*(len(already_checked)-1)+number_to_card(already_checked[-1]))
                else:
                    result.append("")

                result.extend(display_board(placed_card))
                result.append("")
                
            if already_checked:
                result.append("Cannot place card from top of stack of cards put aside ☹️")
                result.append("")
        else:
            result.append("")
    return already_checked
            

    
def run_one_game(random_seed):
    seed(random_seed)
    cards = sorted(set(range(52)) - {6,19,32,45})
    shuffle(cards)
    
    result = ["All 7s removed and placed, rest of deck shuffled, ready to start!"]
    result.append("]" * len(cards))

    placed_card = [["", "", "", ""] for _ in range(13)]
    placed_card[6] = [number_to_card(19),number_to_card(32),number_to_card(45),number_to_card(6)]
    waiting_to_match = [5,7,18,20,31,33,44,46]
    result.append("")
    result.extend(display_board(placed_card))

    for number in range(1,4):
        result.append("")
        already_checked = run_one_round(number, result, waiting_to_match, cards, placed_card)
        if already_checked == []:
            break
        else:
            cards = already_checked[::-1]
            
    if waiting_to_match == []:
        result.append("You placed all cards, you won 👍")
    else:
        result.append(f"You could not place {len(already_checked)} cards, you lost 👎")
    return result, len(already_checked)

def simulate(times, i):
    all_results = {}
    for time in range(times):
        result, left_card_num = run_one_game(i + time)
        all_results[left_card_num] = all_results.get(left_card_num, 0) + 1
    print("Number of cards left | Frequency")
    print("--------------------------------")
    for number in sorted(all_results, reverse = True):
        frequency = all_results[number] / times
        print(f"{number:33} | {frequency:9.2%}")

def is_valid_number(str_number):
    str_number=str_number.strip("")
    if str_number == "":
        return False
    if str_number[0]=="-":
        str_number = str_number[1:]
    return str_number.isdigit()
    
if __name__ == "__main__":
    random_seed = int(input("Please enter an integer to feed the seed() function: "))
    result, left_card_num = run_one_game(random_seed)

    line_number = len(result)
    print(f"\nThere are {line_number} lines of output; what do you want me to do?\n")
    while True:
        print("Enter: q to quit")
        print(f"       a last line number (between 1 and {line_number})")
        print(f"       a first line number (between -1 and -{line_number})")
        print(f"       a range of line numbers (of the form m--n with 1 <= m <= n <= {line_number})")
        user_input = input("       ")
        need_to_print = []
        if user_input == "q":
            break
        else:
            lst = user_input.split("--")
            if len(lst)== 2 and is_valid_number(lst[0]) and is_valid_number(lst[1]):
                m,n=int(lst[0]),int(lst[1])
                if 1 <= m<= n<= line_number:
                    need_to_print=result[m-1:n]
            elif len(lst)==1 and is_valid_number(lst[0]):
                number = int(lst[0])
                if 1 <= number <= line_number:
                    need_to_print= result[:number]
                elif -line_number<= number <= 1:
                    need_to_print=result[number:]
        if need_to_print:
            print()
            for line in need_to_print:
                print(line.rstrip())
        print()

Please enter an integer to feed the seed() function:  1



There are 1048 lines of output; what do you want me to do?

Enter: q to quit
       a last line number (between 1 and 1048)
       a first line number (between -1 and -1048)
       a range of line numbers (of the form m--n with 1 <= m <= n <= 1048 )


        1 -- 100



Enter: q to quit
       a last line number (between 1 and 1048)
       a first line number (between -1 and -1048)
       a range of line numbers (of the form m--n with 1 <= m <= n <= 1048 )


        1--100



All 7s removed and placed, rest of deck shuffled, ready to start!
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]






	🃙	🂱	🂻	🂽






Starting First round...

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
🃇

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
[🂾

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
[[🂺

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
[[[🃗

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
[[[[🃁

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
[[[[[🂸

Cannot place card from top of stack of cards put aside ☹️
]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
[[[[[[🂭

Placing card from top of stack of cards left 😊️
]]]]]]]]]]]]]]]]]]]]]]]]]]]

        q


In [45]:
# for line in result:
#     print(line)
len(result)

1048

In [47]:
simulate(400,11)

Number of cards left | Frequency
--------------------------------
                               11 |     0.50%
                               10 |     0.50%
                                9 |     2.50%
                                8 |     2.50%
                                7 |     4.50%
                                6 |     4.75%
                                5 |     7.25%
                                4 |     7.75%
                                3 |     5.50%
                                2 |     3.75%
                                0 |    60.50%


In [ ]:
from random import seed, shuffle
from collections import defaultdict

class SolitaireGame:
    @staticmethod
    def number_to_card(card_number):
        """Convert card number to Unicode playing card character."""
        # Map suits to Unicode base values (Spades, Hearts, Diamonds, Clubs)
        suit_to_unicode = defaultdict(lambda: 0x1F0B1, {
            0: 0x1F0B1,  # Spades
            1: 0x1F0C1,  # Hearts
            2: 0x1F0D1,  # Diamonds
            3: 0x1F0A1   # Clubs
        })
        # Rank offsets for A,7,8,9,10,J,Q,K (note no 2-6 in this game)
        rank_offsets = [0, 6, 7, 8, 9, 10, 12, 13]  
        suit, rank = card_number // 8, card_number % 8  # 8 ranks per suit
        return chr(suit_to_unicode[suit] + rank_offsets[rank])
    
    @staticmethod
    def display_board(game_board):
        """Format the game board for display."""
        return ["\t" + "\t".join(row) for row in game_board]
    
    @staticmethod
    def place_card(game_board, playable_cards, card_number):
        """Update game state after placing a card."""
        playable_cards.remove(card_number)
        row, col = 12 - card_number % 13, card_number // 13 - 1
        game_board[row][col] = SolitaireGame.number_to_card(card_number)
        
        # Add adjacent cards to playable list based on position
        if 0 < row < 6:
            playable_cards.append(card_number + 1)  # Next higher card
        elif 6 < row < 12:
            playable_cards.append(card_number - 1)  # Next lower card
    
    @staticmethod
    def play_round(round_number, game_log, playable_cards, deck, game_board):
        """Execute one round of the game."""
        round_names = {1: "First", 2: "Second", 3: "Third"}
        game_log.append(f"Starting {round_names[round_number]} round...")
        game_log.append("")
        
        unplayable_cards = []  # Cards that couldn't be placed
        for _ in reversed(deck):  # Process cards from top of deck
            current_card = deck.pop()
            
            if current_card in playable_cards:
                game_log.append("Placing card from top of stack of cards left 😊️")
            else:
                game_log.append("Cannot place card from top of stack of cards put aside ☹️")
                unplayable_cards.append(current_card)
                
            # Display current deck state
            game_log.append("]" * len(deck))
            if unplayable_cards:
                game_log.append("[" * (len(unplayable_cards) - 1) + 
                              SolitaireGame.number_to_card(unplayable_cards[-1]))
            else:
                game_log.append("")
                
            if current_card in playable_cards:
                SolitaireGame.place_card(game_board, playable_cards, current_card)
                game_log.extend(SolitaireGame.display_board(game_board))
                game_log.append("")
                
                # Check if any previously unplayable cards can now be placed
                while unplayable_cards and unplayable_cards[-1] in playable_cards:
                    game_log.append("Placing card from top of stack of cards put aside 😊️")
                    game_log.append("]" * len(deck))
                    SolitaireGame.place_card(game_board, playable_cards, unplayable_cards.pop())
                    
                    if unplayable_cards:
                        game_log.append("[" * (len(unplayable_cards) - 1) + 
                                      SolitaireGame.number_to_card(unplayable_cards[-1]))
                    else:
                        game_log.append("")
                        
                    game_log.extend(SolitaireGame.display_board(game_board))
                    game_log.append("")
                    
                if unplayable_cards:
                    game_log.append("Cannot place card from top of stack of cards put aside ☹️")
                    game_log.append("")
            else:
                game_log.append("")
                
        return unplayable_cards

    @staticmethod
    def play_game(seed_value):
        """Run a complete game with the given random seed."""
        seed(seed_value)
        # Initialize deck (52 cards minus the four 7s)
        deck = sorted(set(range(52)) - {6, 19, 32, 45})
        shuffle(deck)
        
        game_log = [
            "All 7s removed and placed, rest of deck shuffled, ready to start!",
            "]" * len(deck)
        ]
        
        # Initialize empty board (13 rows, 4 columns)
        game_board = [["", "", "", ""] for _ in range(13)]
        # Place the four 7s in the middle row
        game_board[6] = [
            SolitaireGame.number_to_card(19),  # Hearts 7
            SolitaireGame.number_to_card(32),  # Diamonds 7
            SolitaireGame.number_to_card(45),  # Clubs 7
            SolitaireGame.number_to_card(6)    # Spades 7
        ]
        
        # Cards adjacent to the 7s that can be placed next
        playable_cards = [5, 7, 18, 20, 31, 33, 44, 46]
        
        game_log.append("")
        game_log.extend(SolitaireGame.display_board(game_board))
        
        # Play up to 3 rounds
        for round_number in range(1, 4):
            game_log.append("")
            unplayable_cards = SolitaireGame.play_round(
                round_number, game_log, playable_cards, deck, game_board
            )
            if not unplayable_cards:  # All cards placed
                break
            else:  # Prepare for next round with remaining cards
                deck = unplayable_cards[::-1]
                
        # Game conclusion
        if not playable_cards:
            game_log.append("You placed all cards, you won 👍")
        else:
            game_log.append(f"You could not place {len(unplayable_cards)} cards, you lost 👎")
            
        return game_log, len(unplayable_cards)
    
    @staticmethod
    def run_simulations(num_simulations, starting_seed):
        """Run multiple game simulations and show statistics."""
        results = {}  # Track frequency of each outcome
        
        for simulation in range(num_simulations):
            _, remaining_cards = SolitaireGame.play_game(starting_seed + simulation)
            results[remaining_cards] = results.get(remaining_cards, 0) + 1
            
        # Print simulation results
        print("Number of cards left | Frequency")
        print("--------------------------------")
        for cards_left in sorted(results, reverse=True):
            frequency = results[cards_left] / num_simulations
            print(f"{str(cards_left).rjust(20)} | {frequency:9.2%}")

# Create a direct simulate() function
def simulate(num_games=500, start_seed=11):
    """Convenience function to run simulations.
    
    Args:
        num_games (int): Number of games to simulate (default 500)
        start_seed (int): Starting seed value (default 11)
    """
    SolitaireGame.run_simulations(num_games, start_seed)

if __name__ == "__main__":
    # Main game execution
    seed_value = int(input("Please enter an integer to feed the seed() function: "))
    game_log, remaining_cards = SolitaireGame.play_game(seed_value)
    log_length = len(game_log)
    
    print(f"\nThere are {log_length} lines of output; what do you want me to do?\n")
    
    # User interaction loop
    while True:
        print("Enter: q to quit")
        print(f"       a last line number (between 1 and {log_length})")
        print(f"       a first line number (between -1 and -{log_length})")
        print(f"       a range of line numbers (of the form m--n with 1 <= m <= n <= {log_length})")
        
        user_input = input("       ")
        output_to_show = []
        
        if user_input == "q":
            break
        else:
            parts = user_input.split("--")
            
            # Handle range input (m--n)
            if (len(parts) == 2 and 
                SolitaireGame.is_valid_number(parts[0]) and 
                SolitaireGame.is_valid_number(parts[1])):
                start, end = int(parts[0]), int(parts[1])
                if 1 <= start <= end <= log_length:
                    output_to_show = game_log[start-1:end]
                    
            # Handle single number input
            elif len(parts) == 1 and SolitaireGame.is_valid_number(parts[0]):
                line_number = int(parts[0])
                if 1 <= line_number <= log_length:
                    output_to_show = game_log[:line_number]
                elif -log_length <= line_number <= 1:
                    output_to_show = game_log[line_number:]
                    
        # Print requested output
        if output_to_show:
            print()
            for line in output_to_show:
                print(line.rstrip())
        print()